# Fine Tune [Stable Diffusion Instruct-Pix2Pix](https://huggingface.co/docs/diffusers/training/instructpix2pix)

For this fine tuning, In have used [AWS Sagemaker](https://aws.amazon.com/sagemaker/) with the following instance:

* Sagemaker Instance: `ml.g5.2xlarge`
* Sagemaker Image: Sagemaker Distribution `1.4`, `1.5`
* Region: [us-east-1](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-regions-availability-zones.html) (_North Virginia_)
* GPU: 1 x [NVIDIA A10G](https://aws.amazon.com/blogs/aws/new-ec2-instances-g5-with-nvidia-a10g-tensor-core-gpus/)
* VRAM: 24 GB
* Storage: 90 GB

## `nvidia-smi` output on IDLE

In [7]:
! nvidia-smi

Wed Mar 27 10:50:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   21C    P0              52W / 300W |      4MiB / 23028MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

__STEP 1:__ Setup runtime

__STEP 1.1:__ Clone the [huggingface/diffusers](https://github.com/huggingface/diffusers) Github repository. It is recommended to use a specific version of the repository to avoid downloading a pre-release build. To see all the available versions, check out all the available [tags](https://github.com/huggingface/diffusers/tags). Then use the following command to clone the repository at that particular release version. 

In the following command, the `v0.27.0` tag is used which implies that diffusers v0.27.0 is being used. 

In [ ]:
! git clone --branch v0.27.0 https://github.com/huggingface/diffusers.git

__STEP 1.2:__ Install `diffusers` from source code.

In [ ]:
! python3 -m pip install -e ./diffusers

__STEP 1.3:__ Install other dependencies

In [ ]:
! python3 -m pip install --no-cache-dir xformers transformers[torch] ftfy datasets tensorboard

__STEP 1.4:__ Configure [accelerate](https://github.com/huggingface/accelerate)

In [ ]:
! accelerate config default

__STEP 2:__ Login to [huggingface hub](https://hf.co)

Obtain a _Read/Write_ or _Full Access_ Access Token by following the steps:

1. Login to your huggingface account.
2. Go to _Settings -> Access Tokens -> New Token_
3. Click on _New Token_. Provde a _Name_ for your key and select the Type as _Write_ for Full Access.
4. Copy the Key 

In [ ]:
! huggingface-cli login --token=<paste-your-token>

__STEP 5:__ Start Training

In [ ]:
! export BASE_MODEL=stabilityai/stable-diffusion-2-1
! export DATASET=<your-dataset>
! export TUNED_MODEL=<tuned-model-name>

In [ ]:
! accelerate launch --mixed_precision="fp16" \
    ./diffusers/examples/instruct_pix2pix/train_instruct_pix2pix.py \
    --pretrained_model_name_or_path=${BASE_MODEL} \
    --dataset_name=${DATASET} \
    --enable_xformers_memory_efficient_attention \
    --resolution=512 --random_flip \
    --train_batch_size=2 --gradient_accumulation_steps=4 --gradient_checkpointing \
    --max_train_steps=100 \
    --checkpointing_steps=25 --checkpoints_total_limit=1 \
    --learning_rate=5e-05 --max_grad_norm=1 --lr_warmup_steps=20 \
    --conditioning_dropout_prob=0.1 \
    --mixed_precision=fp16 \
    --seed=42 \
    --push_to_hub \
    --report_to=tensorboard \
    --output_dir=${TUNED_MODEL} \
    --original_image_column=before \
    --edit_prompt=prompt \
    --edited_image=after